# 10分钟改变大模型自我认知，定制“专属自己”的聊天机器人

我们使用ms-swift对Qwen2.5-3B-Instruct进行自我认知微调。

- 模型：https://modelscope.cn/models/Qwen/Qwen2.5-3B-Instruct

- 自我认知数据集：https://modelscope.cn/datasets/swift/self-cognition

- 训练框架：https://github.com/modelscope/ms-swift.git

- 实验环境：A10、3090等（需显存资源12GB）

这里给出了两种训练和推理的方式，分别是：使用命令行界面和使用Python。

- 使用命令行界面：帮助开发者更快的将训练和推理跑起来。

- 使用Python：帮助开发者了解训练和推理的一些细节，这对定制训练过程有很大帮助。

准备好了吗？让我们开始这段旅程叭……

## 安装 ms-swift

In [ ]:
%pip install ms-swift -U
%pip install transformers -U

## 使用Python


#### 训练
导入一些库：

In [1]:
# 导入操作系统相关的库
import os
# 设置CUDA可见设备为第0块GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 从swift.llm模块导入模型和分词器加载函数
# get_model_tokenizer: 获取模型和分词器
# load_dataset: 加载数据集
# get_template: 获取对话模板
# EncodePreprocessor: 文本编码预处理器
from swift.llm import get_model_tokenizer, load_dataset, get_template, EncodePreprocessor

# 从swift.utils模块导入工具函数
# get_logger: 获取日志记录器
# get_model_parameter_info: 获取模型参数信息
# plot_images: 绘制图像
# seed_everything: 设置随机种子确保实验可重复
from swift.utils import get_logger, get_model_parameter_info, plot_images, seed_everything

# 从swift.trainers模块导入训练器和训练参数
# Seq2SeqTrainer: 序列到序列训练器
# Seq2SeqTrainingArguments: 序列到序列训练参数配置
from swift.trainers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# 从functools导入partial函数，用于创建偏函数
from functools import partial

# 获取日志记录器实例
logger = get_logger()
# 设置全局随机种子为42，确保实验结果可重复
seed_everything(42)

# 注意：已移除LoRA相关导入
# 不再需要: from swift.tuners import Swift, LoraConfig, find_all_linears

/home/work/hd/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/home/work/hd/.venv/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Successfully registered `/home/work/hd/.venv/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.
This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=

This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=

[INFO:swift] Global seed set to 42

42

设置训练的超参数：

In [7]:
# ========== 模型配置 ==========
# 指定要使用的模型路径或模型ID
model_id_or_path = '/home/work/hd/_models/base/qwen3-4b-thinking'  # model_id or model_path
# 设置系统提示语，定义AI助手的基本行为
system = 'You are a helpful assistant.'
# 设置训练输出目录，保存模型检查点和日志
output_dir = 'output'

# ========== 数据集配置 ==========
# 定义训练数据集列表，#后的数字表示使用该数据集的前N条数据
# AI-ModelScope/alpaca-gpt4-data-zh: 中文指令数据集，使用前500条
# AI-ModelScope/alpaca-gpt4-data-en: 英文指令数据集，使用前500条  
# swift/self-cognition: 自我认知数据集，使用前500条
dataset = ['AI-ModelScope/alpaca-gpt4-data-zh#500', 'AI-ModelScope/alpaca-gpt4-data-en#500',
           'swift/self-cognition#500']  # dataset_id or dataset_path
# 数据集随机种子，确保数据划分的可重复性
data_seed = 42
# 输入序列的最大长度（tokens数量）
max_length = 2048
# 验证集划分比例，0.01表示1%的数据用作验证集
split_dataset_ratio = 0.01  # 切分验证集
# 数据预处理时使用的进程数，加快数据处理速度
num_proc = 4  # 预处理的进程数

# ========== 自我认知数据集个性化配置 ==========
# 替换自我认知数据集中的填充符：{{NAME}}, {{AUTHOR}}
# 模型的中文名和英文名，用于替换数据集中的{{NAME}}占位符
model_name = ['精衡', 'Jingheng']  # 模型的中文名和英文名
# 模型作者的中文名和英文名，用于替换数据集中的{{AUTHOR}}占位符
model_author = ['叶博韬', 'YE SEVERIN']  # 模型作者的中文名和英文名

# ========== 全参数微调配置 ==========
# 注意：全参数微调将训练模型的所有参数，需要更大的显存和更长的训练时间
# 建议降低学习率和批次大小以避免训练不稳定

# ========== 训练超参数配置 ==========
# 创建序列到序列训练参数对象
training_args = Seq2SeqTrainingArguments(
    # 模型检查点和日志的输出目录
    output_dir=output_dir,
    # 学习率，全参数微调使用更小的学习率
    learning_rate=5e-6,  # 从1e-4降低到5e-6
    # 每个GPU上的训练批次大小，全参数微调需要更小的批次
    per_device_train_batch_size=1,  # 可能需要进一步降低
    # 每个GPU上的验证批次大小
    per_device_eval_batch_size=1,
    # 启用梯度检查点，节省显存但增加计算时间
    gradient_checkpointing=True,
    # 权重衰减系数，防止过拟合的正则化技术
    weight_decay=0.1,
    # 学习率调度器类型，cosine表示余弦退火调度
    lr_scheduler_type='cosine',
    # 预热比例，训练初期逐渐增加学习率的步数占总步数的比例
    warmup_ratio=0.05,
    # 日志记录工具，tensorboard用于可视化训练过程
    report_to=['tensorboard'],
    # 是否在第一步就开始记录日志
    logging_first_step=True,
    # 模型保存策略，'steps'表示按步数保存
    save_strategy='steps',
    # 每隔50步保存一次模型检查点
    save_steps=50,
    # 模型评估策略，'steps'表示按步数评估
    eval_strategy='steps',
    # 每隔50步进行一次模型评估
    eval_steps=50,
    # 梯度累积步数，可能需要增加以补偿更小的批次大小
    gradient_accumulation_steps=32,  # 从16增加到32
    # 训练轮数，全参数微调通常需要更少的epoch
    num_train_epochs=1,
    # 选择最佳模型的评估指标，'loss'表示以损失函数为准
    metric_for_best_model='loss',
    # 最多保存的检查点数量，超过后删除最旧的
    save_total_limit=2,
    # 每隔5步记录一次训练日志
    logging_steps=5,
    # 数据加载器使用的工作进程数
    dataloader_num_workers=1,
    # 数据相关的随机种子
    data_seed=data_seed,
    # 全参数微调专用配置
    # 启用BF16混合精度训练以节省显存（适用于BFloat16模型）
    bf16=True,  # 使用bf16而不是fp16，适配BFloat16模型
    # 设置最大梯度范数，防止梯度爆炸
    max_grad_norm=1.0,
    # 禁用FP16（因为我们使用BF16）
    fp16=False,
)

# 将输出目录转换为绝对路径
output_dir = os.path.abspath(os.path.expanduser(output_dir))
# 记录输出目录的绝对路径
logger.info(f'output_dir: {output_dir}')

[INFO:swift] output_dir: /home/work/hd/2_core/1_train/output


获取模型和对话template，进行全参数微调：

In [8]:
# ========== 模型和分词器加载 ==========
# 加载指定路径的模型和分词器，指定模型类型为qwen3_thinking
# model_type参数用于区分同一目录下的不同模型变体
model, tokenizer = get_model_tokenizer(model_id_or_path, model_type='qwen3_thinking')
# 打印模型的基本信息（架构、参数量等）
logger.info(f'model_info: {model.model_info}')

# ========== 对话模板配置 ==========
# 获取模型对应的对话模板，用于格式化输入输出
# model.model_meta.template: 模型元数据中定义的模板类型
# default_system: 设置默认的系统提示语
# max_length: 限制输入序列的最大长度
template = get_template(model.model_meta.template, tokenizer, default_system=system, max_length=max_length)
# 将模板设置为训练模式，这会影响模板的行为（如是否包含特殊token）
template.set_mode('train')

# ========== 全参数微调配置 ==========
# 注意：已移除LoRA相关配置，现在将进行全参数微调
# 全参数微调会训练模型的所有权重，而不仅仅是添加的适配器

# 不再需要以下LoRA相关代码：
# - from swift.tuners import Swift, LoraConfig
# - target_modules = find_all_linears(model)
# - lora_config = LoraConfig(...)
# - model = Swift.prepare_model(model, lora_config)

# ========== 模型信息输出 ==========
# 打印完整的模型结构（全参数微调版本，无LoRA层）
logger.info(f'model: {model}')
# 获取并打印模型参数统计信息（总参数量、可训练参数量等）
model_parameter_info = get_model_parameter_info(model)
logger.info(f'model_parameter_info: {model_parameter_info}')

# ========== 显存优化设置 ==========
# 对于全参数微调，建议启用以下优化设置
# 1. 启用梯度检查点（已在training_args中设置）
# 2. 启用FP16混合精度训练（已在training_args中设置）
# 3. 确保模型在训练前处于正确状态
model.train()  # 确保模型处于训练模式

[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.04it/s]
[INFO:swift] model_info: ModelInfo(model_type='qwen3_thinking', model_dir='/home/work/hd/_models/base/qwen3-4b-thinking', torch_dtype=torch.bfloat16, max_model_len=262144, quant_method=None, quant_bits=None, rope_scaling=None, is_moe_model=False, config=Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "ful

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layer

下载并载入数据集，并切分成训练集和验证集，

然后将文本编码成tokens：

In [4]:
# ========== 数据集加载和划分 ==========
# 加载指定的数据集并自动划分为训练集和验证集
# dataset: 数据集列表，包含中文、英文指令数据和自我认知数据
# split_dataset_ratio: 验证集比例（0.01 = 1%）
# num_proc: 并行处理进程数，加快数据加载速度
# model_name/model_author: 用于替换自我认知数据集中的{{NAME}}和{{AUTHOR}}占位符
# seed: 随机种子，确保数据划分的可重复性
train_dataset, val_dataset = load_dataset(dataset, split_dataset_ratio=split_dataset_ratio, num_proc=num_proc,
        model_name=model_name, model_author=model_author, seed=data_seed)

# ========== 数据集信息输出 ==========
# 打印训练集的基本信息（数据条数、格式等）
logger.info(f'train_dataset: {train_dataset}')
# 打印验证集的基本信息
logger.info(f'val_dataset: {val_dataset}')
# 打印训练集第一条原始数据样本，查看数据格式和内容
logger.info(f'train_dataset[0]: {train_dataset[0]}')

# ========== 数据编码预处理 ==========
# 创建编码预处理器，将文本转换为模型可理解的token序列
# template: 使用之前配置的对话模板来格式化数据
# 对训练集进行编码处理：文本 -> token IDs
train_dataset = EncodePreprocessor(template=template)(train_dataset, num_proc=num_proc)
# 对验证集进行编码处理：文本 -> token IDs
val_dataset = EncodePreprocessor(template=template)(val_dataset, num_proc=num_proc)
# 打印编码后的训练集第一条数据，查看token化后的格式
logger.info(f'encoded_train_dataset[0]: {train_dataset[0]}')

# ========== 样本展示 ==========
# 使用模板的打印功能展示一条完整的训练样本
# 这会显示格式化后的输入输出对，包括系统提示、用户输入、模型回答等
template.print_inputs(train_dataset[0])

[INFO:swift] SelfCognitionPreprocessor has been successfully configured with name: ['精衡', 'Jingheng'], author: ['叶博韬', 'YE SEVERIN'].
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-zh
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-zh
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-en
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-en
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: swift/self-cognition
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: swift/self-cognition
[WARNING:swift] dataset_sample:499 is greater than len(dataset):107, repeated sampling will be performed.
[INFO:swift] train_dataset: Dataset({
    features: ['messages'],
    num_rows: 1489
})
[INFO:swift] val_dataset: Dataset({
    features: ['messages'],
    num_rows: 11
})
[WARNING:swift

初始化trainer并开始训练：

In [ ]:
# ========== 模型训练前准备 ==========
# 启用输入梯度计算，这是使用梯度检查点（gradient checkpointing）时的必要设置
# 梯度检查点可以节省显存，但需要在前向传播时重新计算某些中间激活值
model.enable_input_require_grads()  # 兼容gradient checkpointing

# ========== 训练器初始化 ==========
# 创建序列到序列训练器对象，配置所有训练相关组件
trainer = Seq2SeqTrainer(
    model=model,                        # 要训练的模型（全参数微调版本）
    args=training_args,                 # 训练参数配置（学习率、批次大小、保存策略等）
    data_collator=template.data_collator,  # 数据整理器，用于将批次数据转换为模型输入格式
    train_dataset=train_dataset,        # 训练数据集（已编码为token序列）
    eval_dataset=val_dataset,           # 验证数据集（用于评估模型性能）
    template=template,                  # 对话模板（用于格式化输入输出）
)

# ========== 开始训练 ==========
# 启动训练过程，这将执行以下步骤：
# 1. 前向传播：计算模型输出和损失
# 2. 反向传播：计算梯度
# 3. 参数更新：使用优化器更新所有模型权重（全参数微调）
# 4. 定期保存检查点和记录日志
# 5. 定期在验证集上评估模型性能
trainer.train()

# ========== 训练完成后处理 ==========
# 获取最后保存的模型检查点路径
# trainer.state.last_model_checkpoint 包含了训练过程中最后一次保存的模型路径
last_model_checkpoint = trainer.state.last_model_checkpoint
# 记录最终模型检查点的位置，用于后续推理
logger.info(f'last_model_checkpoint: {last_model_checkpoint}')

The model is already on multiple devices. Skipping the move to device specified in `args`.
Train:   2%|▏         | 1/47 [00:05<04:30,  5.89s/it] 

{'loss': 1.69011736, 'grad_norm': 5.96875, 'learning_rate': 1.67e-06, 'token_acc': 0.57945684, 'epoch': 0.02, 'global_step/max_steps': '1/47', 'percentage': '2.13%', 'elapsed_time': '5s', 'remaining_time': '4m 32s', 'memory(GiB)': 60.49, 'train_speed(iter/s)': 0.169098}


Train:   4%|▍         | 2/47 [00:11<04:22,  5.84s/it]

可视化训练的loss。其中浅黄色线条代表真实loss值，黄色线条代表经过0.9平滑系数平滑后的loss值。

你也可以使用tensorboard进行实时可视化，在命令行输入`tensorboard --logdir '{output_dir}/runs'`。

In [ ]:
images_dir = os.path.join(output_dir, 'images')
logger.info(f'images_dir: {images_dir}')
plot_images(images_dir, training_args.logging_dir, ['train/loss'], 0.9)  # 保存图片

# 展示图片
from IPython.display import display
from PIL import Image
image = Image.open(os.path.join(images_dir, 'train_loss.png'))
display(image)

#### 微调后推理

导入一些库：

In [ ]:
# ========== 推理环境设置 ==========
# 导入操作系统相关库
import os
# 设置CUDA可见设备为第0块GPU（与训练时保持一致）
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# ========== 推理相关库导入 ==========
# 从swift.llm模块导入推理相关的核心类和函数
from swift.llm import (
    InferEngine,      # 推理引擎基类，定义推理接口
    InferRequest,     # 推理请求对象，封装用户输入和配置
    PtEngine,         # PyTorch推理引擎，用于加载和运行PyTorch模型
    RequestConfig,    # 请求配置对象，设置生成参数（温度、最大长度等）
    get_template      # 获取对话模板函数，用于格式化输入输出
)

设置推理的超参数：

In [ ]:
# ========== 微调后模型路径配置 ==========
# 指定训练完成后保存的模型检查点路径
# 注意：全参数微调保存的是完整的模型权重，不是LoRA适配器
# 格式为 output/vx-xxx/checkpoint-xxx
last_model_checkpoint = 'output/vx-xxx/checkpoint-xxx'

# ========== 推理模型配置 ==========
# 对于全参数微调，直接使用微调后的完整模型路径
# 不再需要指定基础模型 + LoRA适配器的组合方式
model_id_or_path = last_model_checkpoint  # 直接使用微调后的完整模型
# 设置系统提示语，定义AI助手的行为特征
system = 'You are a helpful assistant.'
# 指定推理后端类型，'pt'表示使用PyTorch后端
infer_backend = 'pt'

# ========== 文本生成参数配置 ==========
# 设置生成文本的最大新token数量（不包括输入部分）
max_new_tokens = 10000
# 设置生成温度，控制文本生成的随机性
# temperature=0: 确定性生成，总是选择概率最高的token
# temperature>0: 随机性生成，值越大越随机
temperature = 0
# 是否启用流式输出，True表示逐token生成并实时返回
# 流式输出可以提供更好的用户体验，类似ChatGPT的打字效果
stream = True

获取推理引擎，载入全参数微调后的完整模型：

In [ ]:
# ========== 推理引擎初始化 ==========
# 创建PyTorch推理引擎，直接加载微调后的完整模型
# 全参数微调后，模型的所有权重都已经更新，不需要额外的适配器
# model_id_or_path: 微调后的完整模型路径
engine = PtEngine(model_id_or_path)  # 不再需要adapters参数

# ========== 对话模板配置 ==========
# 获取与推理引擎模型匹配的对话模板
# engine.model.model_meta.template: 从模型元数据中获取模板类型
# engine.tokenizer: 使用推理引擎的分词器
# default_system: 设置默认的系统提示语
template = get_template(engine.model.model_meta.template, engine.tokenizer, default_system=system)

# ========== 设置默认模板 ==========
# 将配置好的模板设置为推理引擎的默认模板
# 这样在调用engine.infer()时会自动使用此模板格式化输入输出
# 也可以在每次调用engine.infer()时单独传入template参数
engine.default_template = template

开始推理...

In [ ]:
query_list = [
    'who are you?',
    "晚上睡不着觉怎么办？",
    '你是谁训练的？',
]

def infer_stream(engine: InferEngine, infer_request: InferRequest):
    request_config = RequestConfig(max_tokens=max_new_tokens, temperature=temperature, stream=True)
    gen_list = engine.infer([infer_request], request_config)
    query = infer_request.messages[0]['content']
    print(f'query: {query}\nresponse: ', end='')
    for resp in gen_list[0]:
        if resp is None:
            continue
        print(resp_list[0].choices[0].delta.content, end='', flush=True)
    print()

def infer(engine: InferEngine, infer_request: InferRequest):
    request_config = RequestConfig(max_tokens=max_new_tokens, temperature=temperature)
    resp_list = engine.infer([infer_request], request_config)
    query = infer_request.messages[0]['content']
    response = resp_list[0].choices[0].message.content
    print(f'query: {query}')
    print(f'response: {response}')

infer_func = infer_stream if stream else infer
for query in query_list:
    infer_func(engine, InferRequest(messages=[{'role': 'user', 'content': query}]))
    print('-' * 50)

## Web-UI

In [ ]:
# ========== Web-UI启动命令 ==========
# 使用全参数微调后的完整模型启动Web界面
# 注意：需要将 output/vx-xxx/checkpoint-xxx 替换为实际的检查点路径

# 方法1：查看实际的检查点路径
print(f"实际的检查点路径: {last_model_checkpoint}")

# 方法2：启动Web-UI（需要替换为实际路径）
# 全参数微调后，直接使用完整模型路径，不需要指定适配器
!CUDA_VISIBLE_DEVICES=0 \
swift web-ui \
    --model_id_or_path {last_model_checkpoint} \
    --model_type qwen3_thinking \
    --temperature 0 \
    --infer_backend pt \
    --max_new_tokens 2048

# 注意：全参数微调版本不需要 --adapters 参数
# 因为模型的所有权重都已经在微调过程中更新了